In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np

def normalize(x):
    """
    Normalize a list of sample image data in the range of 0 to 1
    : x: List of image data.  The image shape is (32, 32, 3)
    : return: Numpy array of normalize data
    """
    # TODO: Implement Function
    from numpy import linalg as LA
    print (x.shape)
    x=x/255
    print (x.shape)
    return x

print("done")

done


In [2]:
map=[0,1,2,3,4,5,6,7,8,9]
def one_hot_encode(x):
    """
    One hot encode a list of sample labels. Return a one-hot encoded vector for each label.
    : x: List of sample Labels
    : return: Numpy array of one-hot encoded labels
    """
    # TODO: Implement Function
    from sklearn import preprocessing
    lb=preprocessing.LabelBinarizer()
    lb.fit(map)
    
    return lb.transform(x)
print("done")

done


In [3]:
epochs = 25
batch_size = 2
keep_probability = 0.9
read_threads=1
print("done")

done


In [4]:
import tensorflow as tf

def neural_net_image_input(image_shape):
    """
    Return a Tensor for a batch of image input
    : image_shape: Shape of the images
    : return: Tensor for image input.
    """
    # TODO: Implement Function
    
    t=tf.placeholder(tf.float32,shape=[None,image_shape[0],image_shape[1],image_shape[2]],name='x')
    return t


def neural_net_label_input(n_classes):
    """
    Return a Tensor for a batch of label input
    : n_classes: Number of classes
    : return: Tensor for label input.
    """
    # TODO: Implement Function
    t=tf.placeholder(tf.int32,shape=[None,n_classes],name='y')
    return t


def neural_net_keep_prob_input():
    """
    Return a Tensor for keep probability
    : return: Tensor for keep probability.
    """
    # TODO: Implement Function
    t=tf.placeholder(tf.float32,name='keep_prob')
    return t


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tf.reset_default_graph()
print("done")

done


In [5]:
def conv2d_maxpool(x_tensor, conv_num_outputs, conv_ksize, conv_strides, pool_ksize, pool_strides):
    """
    Apply convolution then max pooling to x_tensor
    :param x_tensor: TensorFlow Tensor
    :param conv_num_outputs: Number of outputs for the convolutional layer
    :param conv_ksize: kernal size 2-D Tuple for the convolutional layer
    :param conv_strides: Stride 2-D Tuple for convolution
    :param pool_ksize: kernal size 2-D Tuple for pool
    :param pool_strides: Stride 2-D Tuple for pool
    : return: A tensor that represents convolution and max pooling of x_tensor
    """
    # TODO: Implement Function
    from math import sqrt
    batch_size,dim1,dim2,dim3=x_tensor.get_shape().as_list()
    filter_weights=tf.Variable(tf.truncated_normal(list(conv_ksize)+[x_tensor.get_shape().as_list()[-1]]+[conv_num_outputs],stddev=0.1))
    filter_bias=tf.Variable(tf.zeros(conv_num_outputs,dtype=tf.float32))
    strides=[1]+list(conv_strides)+[1]
    padding='SAME'
    conv=tf.nn.conv2d(x_tensor,filter_weights,strides,padding)
    conv=tf.nn.bias_add(conv,filter_bias)
    conv=tf.nn.relu(conv)
    
    return tf.nn.max_pool(conv,ksize=[1]+list(pool_ksize)+[1],strides=[1]+list(pool_strides)+[1],padding='SAME')


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
print("done")

done


In [6]:
def flatten(x_tensor):
    """
    Flatten x_tensor to (Batch Size, Flattened Image Size)
    : x_tensor: A tensor of size (Batch Size, ...), where ... are the image dimensions.
    : return: A tensor of size (Batch Size, Flattened Image Size).
    """
    # TODO: Implement Function
    batch_size,dim1,dim2,dim3=x_tensor.get_shape().as_list()
    newdim=dim1*dim2*dim3
    return tf.reshape(x_tensor,[-1,newdim])
print("done")

done


In [7]:
def fully_conn(x_tensor, num_outputs):
    """
    Apply a fully connected layer to x_tensor using weight and bias
    : x_tensor: A 2-D tensor where the first dimension is batch size.
    : num_outputs: The number of output that the new tensor should be.
    : return: A 2-D tensor where the second dimension is num_outputs.
    """
    # TODO: Implement Function
    from math import sqrt
    num_features=x_tensor.shape[1].value
    weights=tf.Variable(tf.truncated_normal([num_features]+[num_outputs],stddev=0.1))
    bias=tf.Variable(tf.zeros([num_outputs]))
    con=tf.matmul(x_tensor,weights)
    con=tf.add(con,bias)
    con=tf.nn.relu(con)
    
    
    return con
print("done")

done


In [8]:
def output(x_tensor, num_outputs):
    """
    Apply a output layer to x_tensor using weight and bias
    : x_tensor: A 2-D tensor where the first dimension is batch size.
    : num_outputs: The number of output that the new tensor should be.
    : return: A 2-D tensor where the second dimension is num_outputs.
    """
    # TODO: Implement Function
    
    weights=tf.Variable(tf.truncated_normal([x_tensor.get_shape().as_list()[1]]+[num_outputs],stddev=0.1))
    bias=tf.Variable(tf.zeros([num_outputs]))
    out=tf.matmul(x_tensor,weights)
    out=tf.add(out,bias)
    
    return out
print("done")

done


In [9]:
def conv_net(x, keep_prob):
    """
    Create a convolutional neural network model
    : x: Placeholder tensor that holds image data.
    : keep_prob: Placeholder tensor that hold dropout keep probability.
    : return: Tensor that represents logits
    """
    # TODO: Apply 1, 2, or 3 Convolution and Max Pool layers
    #    Play around with different number of outputs, kernel size and stride
    # Function Definition from Above:
    #    conv2d_maxpool(x_tensor, conv_num_outputs, conv_ksize, conv_strides, pool_ksize, pool_strides)
    con1=conv2d_maxpool(x,8,(4,4),(1,1),(2,2),(2,2))
    con1=conv2d_maxpool(con1,16,(4,4),(1,1),(2,2),(2,2))
    
    con1=conv2d_maxpool(con1,32,(4,4),(1,1),(2,2),(2,2))

    # TODO: Apply a Flatten Layer
    # Function Definition from Above:
    #   flatten(x_tensor)
    flat=flatten(con1)
    

    # TODO: Apply 1, 2, or 3 Fully Connected Layers
    #    Play around with different number of outputs
    # Function Definition from Above:
    #   fully_conn(x_tensor, num_outputs)
    
    fc1=fully_conn(flat,1024)    
    fc1=tf.nn.dropout(fc1,keep_prob)
    fc1=fully_conn(fc1,1024)
    fc1=tf.nn.dropout(fc1,keep_prob)
    
    
    
    # TODO: Apply an Output Layer
    #    Set this to the number of classes
    # Function Definition from Above:
    #   output(x_tensor, num_outputs)
    fc1=output(fc1,10)
    
    # TODO: return output
    return fc1


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""


#tests.test_conv_net(conv_net)
print("done")

done


In [10]:
def train_neural_network(session, optimizer, keep_probability, image_batch, label_batch):
    """
    Optimize the session on a batch of images and labels
    : session: Current TensorFlow session
    : optimizer: TensorFlow optimizer function
    : keep_probability: keep probability
    : feature_batch: Batch of Numpy image data
    : label_batch: Batch of Numpy label data
    """
    # TODO: Implement Function
    session.run(cost,feed_dict={x:image_batch,y:label_batch,keep_prob:keep_probability})
    session.run(optimizer,feed_dict={x:image_batch,y:label_batch,keep_prob:keep_probability})
    pass
print("done")

done


In [11]:
def print_stats(session, image_batch, label_batch, cost, accuracy):
    """
    Print information about loss and validation accuracy
    : session: Current TensorFlow session
    : feature_batch: Batch of Numpy image data
    : label_batch: Batch of Numpy label data
    : cost: TensorFlow cost function
    : accuracy: TensorFlow accuracy function
    """
    # TODO: Implement Function
    loss=session.run(cost,feed_dict={x:image_batch,y:label_batch,keep_prob: 1.0})
    acc=session.run(accuracy,feed_dict={x:valid_features,y:valid_labels,keep_prob:1.0})
    print('loss',loss)
    print('accuracy',acc)
    pass
print("done")

done


In [12]:
#Second method called for batching
def read_my_file_format(filename_queue):
        reader = tf.TextLineReader()
        key, record_string = reader.read(filename_queue)
        # "a" means representative value to indicate type for csv cell value.
        image_file_name, label = tf.decode_csv(record_string, [["a"], ["a"]])

        image_jpeg_data = tf.read_file(image_file_name)
        #depth_png_data = tf.read_file(depth_file_name)
        # channels=1 means image is read as gray-scale
        image_decoded = tf.image.decode_jpeg(image_jpeg_data, channels=3)
        
        #some preprocessing on images can be done
        # Randomly crop a [height, width] section of the image.
        distorted_image = tf.random_crop(image_decoded, [512, 512, 3])
        # Randomly flip the image horizontally.
        distorted_image = tf.image.random_flip_left_right(distorted_image)
        distorted_image = tf.image.random_brightness(distorted_image,
                                               max_delta=63)
        distorted_image = tf.image.random_contrast(distorted_image,
                                             lower=0.2, upper=1.8)
        image_decoded = tf.image.per_image_standardization(image_decoded)
        image_decoded.set_shape([512, 512, 3])


        # Subtract off the mean and divide by the variance of the pixels.

        #depth_decoded = tf.image.decode_png(depth_png_data, channels=3)
        #depth_decoded.set_shape([640, 640, 1])
        return image_decoded, label
print("done")

done


In [13]:
#first method to call for batching
def input_pipeline(filenames, batch_size, read_threads, num_epochs=None):
        filename_queue = tf.train.string_input_producer(filenames, num_epochs=num_epochs, shuffle=True)
        min_after_dequeue = 2
        capacity = min_after_dequeue + 3 * batch_size

        example_list = [read_my_file_format(filename_queue) for _ in range(read_threads)]
        example_batch, label_batch = tf.train.shuffle_batch_join(
            example_list, batch_size=batch_size, capacity=capacity,
            min_after_dequeue=min_after_dequeue)
        return example_batch, label_batch
print("done")

done


In [14]:
def prepare_batch():
        # image_batch, depth_batch : Tensor("shuffle_batch_join_14:0", shape=(batch_size, 640, 640, 1), dtype=uint8)
        image_batch, label_batch = input_pipeline(['sample10.csv'], batch_size, read_threads)
        
        return image_batch,label_batch
print("done")

done


In [16]:
##############################
## Build the Neural Network ##
##############################

# Remove previous weights, bias, inputs, etc..
tf.reset_default_graph()

# Inputs
x = neural_net_image_input((512, 512, 3))
y = neural_net_label_input(10)
keep_prob = neural_net_keep_prob_input()

# Model
logits = conv_net(x, keep_prob)

# Name logits Tensor, so that is can be loaded from disk after training
logits = tf.identity(logits, name='logits')

# Loss and Optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.AdamOptimizer().minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')
print("done")

done


In [25]:
init_op = tf.global_variables_initializer()


i = 0
nLoop = 100 # 200
print("Start Learning loop")
epoch_counter=100
with tf.Session() as sess:
    sess.run(init_op)
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord, sess=sess)

    try:
        while not coord.should_stop():
            i += 1
            # Run training steps or whatever
            image_data, label_data = prepare_batch()#this is doubt
            #image_data = image_data.reshape((LearningNetwork.batch_size, LearningNetwork.nPixels))
            #label_data = label_data.reshape((LearningNetwork.batch_size, LearningNetwork.nPixels))
            
            #image_data = scaler.fit_transform(image_data)
            #some preprocessing on images  can be done
            sess.run(optimizer)
            if i == nLoop:
                coord.request_stop()

            #TODO:Split data into groups for cross-validation
            if i % 400 == 0:
                loss_vals = []
                loss_val,summary = sess.run([loss,  tf.summary],
                                                            feed_dict={image_batch:image_data, label_batch:label_data, nn.keep_prob:1.0})
                loss_vals.append(loss_val)
                loss_val = np.sum(loss_vals)
                print ('Step: %d, Loss: %f @ %s' % (i, loss_val, datetime.now().strftime("%Y/%m/%d %H:%M:%S")))
                nn.saver.save(nn.sess, 'learn_session', global_step=i)
                #t_cmp = tf.reshape(t_cmp, [LearningNetwork.batch_size, nn.nFilteredWidth, nn.nFilteredHeight])
                #out = tf.reshape(out, [LearningNetwork.batch_size, nn.nFilteredWidth, nn.nFilteredHeight])
                #if i % 4000 == 0:
                    #draw_image(out.eval(session=nn.sess), t_cmp.eval(session=nn.sess),
                               #LearningNetwork.batch_size, nn.nFilteredWidth, nn.nFilteredHeight)
                    #writer.add_summary(summary, i)


    except tf.errors.OutOfRangeError:
        print('Done training')
        coord.request_stop(e)
    finally:
        # When done, ask the threads to stop.
        coord.request_stop()

    # stop our queue threads and properly close the session
    coord.request_stop()
    coord.join(threads)

sess.close()

print("end")

Start Learning loop
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.NotFoundError'>, /Users/hmakam200/Desktop/ramya_data/84_left.jpeg
	 [[Node: ReadFile = ReadFile[_device="/job:localhost/replica:0/task:0/cpu:0"](DecodeCSV)]]


InvalidArgumentError: Shape [-1,512,512,3] has negative dimensions
	 [[Node: x = Placeholder[dtype=DT_FLOAT, shape=[?,512,512,3], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'x', defined at:
  File "/usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/hmakam200/personal/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/hmakam200/personal/tensorflow/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/hmakam200/personal/tensorflow/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/Us